# Building a Tensorflow predictive model based on housing price data for Shanghai

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

/Users/noelkonagai/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
raw_data = pd.read_csv('priceData.csv')
raw_data = raw_data.dropna(axis = 0, how = 'any')
raw_data.room = raw_data.room.astype('int64')
raw_data.renovation = raw_data.renovation.astype('int64')
raw_data.lvgroom = raw_data.lvgroom.astype('int64')
# raw_data.floor = raw_data.floor.astype('category')

### Getting statistical values for each variable

In [3]:
raw_data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,7642.0,7214.722717,4382.403296,2.000000,3520.500000,6841.500000,11184.500000,15184.000000
sqm,7642.0,107.751112,61.923839,8.000000,64.000000,97.000000,135.000000,731.000000
room,7642.0,2.246009,0.934408,1.000000,2.000000,2.000000,3.000000,11.000000
lvgroom,7642.0,1.593562,0.616656,0.000000,1.000000,2.000000,2.000000,4.000000
year,7642.0,2000.418477,9.864450,1918.000000,1996.000000,2003.000000,2006.000000,2017.000000
renovation,7642.0,2.767208,0.708619,1.000000,2.000000,3.000000,3.000000,4.000000
price,7642.0,12337.277807,11213.236025,1000.000000,5500.000000,9000.000000,15000.000000,200000.000000
lat,7642.0,31.208627,0.086148,30.707683,31.194303,31.221302,31.241236,31.407696
lng,7642.0,121.481631,0.073207,121.103223,121.434194,121.492974,121.534738,121.711090
postal,7642.0,200495.652185,666.868666,200000.000000,200085.000000,200125.000000,201112.000000,215332.000000


In [4]:
from sklearn.model_selection import train_test_split as tts

x_data = raw_data.drop(['price'],axis=1)
y_data = raw_data['price']

x_train, x_test, y_train, y_test = tts(x_data, y_data, test_size=0.3, random_state=101)

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
numeric_columns = ['sqm', 'year', 'lat', 'lng', 'dist_1',]

for item in numeric_columns:
    print(item)
    scaler.fit(x_train[item])
    scaler.fit(x_train[item])
    x_train[item] = scaler.transform(x_train[item])
    x_test[item] = scaler.transform(x_test[item])

sqm
year


/Users/noelkonagai/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
/Users/noelkonagai/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/noelkonagai/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/User

lat
lng


/Users/noelkonagai/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/noelkonagai/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/noelkonagai/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.resha

dist_1


In [6]:
#tf.feature_column.embedding_column('room', dimension = len(raw_data))

f1 = tf.feature_column.numeric_column('sqm')
f2 = tf.feature_column.numeric_column('year')
f3 = tf.feature_column.numeric_column('dist_1')
f4 = tf.feature_column.categorical_column_with_hash_bucket("room", hash_bucket_size=11)
f4 = tf.feature_column.categorical_column_with_hash_bucket(key = 'room', hash_bucket_size = 11, dtype = tf.int64)
f5 = tf.feature_column.categorical_column_with_hash_bucket(key = 'renovation', hash_bucket_size = 5, dtype = tf.int64)
f6 = tf.feature_column.categorical_column_with_hash_bucket(key = 'lvgroom', hash_bucket_size = 5, dtype = tf.int64)
f7 = tf.feature_column.categorical_column_with_hash_bucket(key = 'floor', hash_bucket_size = 3, dtype = tf.string)

embedded_columns = [tf.feature_column.embedding_column(f4, 1),
           tf.feature_column.embedding_column(f5, 1),
           tf.feature_column.embedding_column(f6, 1),
           tf.feature_column.embedding_column(f7, 1)]

#features = tf.parse_example(features = tf.feature_column.make_parse_example_spec(embedded_columns))
#dense_tensor = input_layer(features, embedded_columns)

feature_columns = [f1, f2] + embedded_columns

In [7]:
input_func = tf.estimator.inputs.pandas_input_fn(
    x = x_train,
    y = y_train,
    batch_size = 100,
    num_epochs = 3000,
    shuffle = True
)

In [8]:
model = tf.estimator.DNNRegressor(
    hidden_units = [7,14,28,14,7],
    feature_columns = feature_columns
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/j5/j768nzj554jbgl0ypv8vxk640000gp/T/tmpzdy1hy9u', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11ade8c88>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
model.train(input_fn = input_func, 
            steps = 5000,
           )

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/j5/j768nzj554jbgl0ypv8vxk640000gp/T/tmpzdy1hy9u/model.ckpt.
INFO:tensorflow:loss = 3.36187e+10, step = 1
INFO:tensorflow:global_step/sec: 133.936
INFO:tensorflow:loss = 3.93277e+09, step = 101 (0.748 sec)
INFO:tensorflow:global_step/sec: 245.162
INFO:tensorflow:loss = 3.23867e+09, step = 201 (0.408 sec)
INFO:tensorflow:global_step/sec: 251.683
INFO:tensorflow:loss = 3.54725e+10, step = 301 (0.400 sec)
INFO:tensorflow:global_step/sec: 248.661
INFO:tensorflow:loss = 7.04707e+09, step = 401 (0.400 sec)
INFO:tensorflow:global_step/sec: 237.434
INFO:tensorflow:loss = 2.96994e+09, step = 501 (0.421 sec)
INFO:tensorflow:global_step/sec: 246.036
INFO:tensorflow:loss = 6.6512e+09, step = 601 (0.407 sec)
INFO:tensorflow:global_step/sec: 246.261
INFO:tensorflow:loss = 4.22098e+09, step = 701 (0.406 sec)
INFO:tensorflow:global_step/sec: 248.508
INFO:tensorflow:loss = 4.6672e+09, step = 801 (0.404

In [10]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x = x_test,
      batch_size = 10,
      num_epochs = 1,
      shuffle = False
)

In [11]:
from sklearn.metrics import mean_squared_error

pred_gen = model.predict(predict_input_func)
predictions = list(pred_gen)

final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

print(mean_squared_error(y_test,final_preds)**0.5)

INFO:tensorflow:Restoring parameters from /var/folders/j5/j768nzj554jbgl0ypv8vxk640000gp/T/tmpzdy1hy9u/model.ckpt-5000
5941.43407404
